<a href="https://colab.research.google.com/github/awaab/FraudDetection/blob/main/credit_train_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import (precision_score, 
                             recall_score, 
                             roc_auc_score,
                             roc_curve,
                             auc,
                             precision_recall_curve)
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier, RUSBoostClassifier

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/kaggle/creditcard.csv')
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [ ]:
eps = 0.001
df['log_Ammount'] = np.log(df.pop('Amount')+eps)
df['Time_round_24']  =df.pop('Time') % (24 * 60 * 60)
#df.pop('Time')

In [ ]:
y = df["Class"].values
df.pop("Class")
X = df.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
def show_scores(X, y, model, plot_roc=False):
  y_pred = model.predict(X)
  print("recall",recall_score(y,y_pred))
  print("percision",precision_score(y,y_pred))
  print("roc_auc",roc_auc_score(y,y_pred))
  percision, recall, thresh = precision_recall_curve(y, y_pred)
  print("pr_auc",auc(percision, recall))

In [37]:
precision_recall_curve

def fig_add_pr(name, labels, predictions, fig):
  percision, recall, thresh = precision_recall_curve(labels, predictions)
  fig.add_trace(go.Scatter(x=100*recall, y=100*percision,
                        mode="lines",#+text",
                        text=thresh,
                        name=name,
                        marker=dict(
                          size=2,
                        )
  )
  )

def plot_pr(name_lis, labels, pred_lis):
  fig = go.Figure()
  for name, predictions in zip(name_lis, pred_lis):
    fig_add_pr(name, labels, predictions, fig)
  fig.update_layout(
      xaxis_title="Recall",
      yaxis_title="Precision"
  )
  fig.show()

In [ ]:
def fig_add_roc(name, labels, predictions, fig):
  fp, tp, thresh = roc_curve(labels, predictions)
  assert len(thresh) == len(tp)
  fig.add_trace(go.Scatter(x=100*fp, y=100*tp,
                        mode="lines",#+text",
                        text=thresh,
                        name=name,
                        marker=dict(
                          size=2,
                        )
  )
  )

def plot_roc(name_lis, labels, pred_lis):
  fig = go.Figure()
  for name, predictions in zip(name_lis, pred_lis):
    fig_add_roc(name, labels, predictions, fig)
  fig.update_layout(
      yaxis_range=[80,100],
      xaxis_range=[0,20],
      xaxis_title="False positives [%]",
      yaxis_title="True positives [%]"
  )
  fig.show()

In [28]:
print("Bagging Classifier")
bagging_model = BaggingClassifier(n_estimators=10)
bagging_model.fit(X_train,y_train)
show_scores(X_val,y_val,bagging_model)

Bagging Classifier
recall 0.8133333333333334
percision 0.9384615384615385
roc_auc 0.9066227048255449
pr_auc 0.8744051933641347


In [29]:
print("Random forest")
rf_model = RandomForestClassifier(n_estimators=10)
rf_model.fit(X_train,y_train)
show_scores(X_val,y_val,rf_model)

Random forest
recall 0.8533333333333334
percision 0.9411764705882353
roc_auc 0.9266227048255448
pr_auc 0.895729742312778


In [30]:
print("Random forest class weighting")
rf_cls_weight_model = RandomForestClassifier(n_estimators=10, class_weight='balanced')
rf_cls_weight_model.fit(X_train,y_train)
show_scores(X_val,y_val,rf_cls_weight_model)

Random forest class weighting
recall 0.8133333333333334
percision 0.9682539682539683
roc_auc 0.9066446857461058
pr_auc 0.8893014082603496


In [31]:
print("Random forest bootstrapped class wieghting")
rf_bts_cls_weight_model = RandomForestClassifier(n_estimators=10, class_weight='balanced_subsample')
rf_bts_cls_weight_model.fit(X_train,y_train)
show_scores(X_val,y_val,rf_bts_cls_weight_model)

Random forest bootstrapped class wieghting
recall 0.8533333333333334
percision 0.9696969696969697
roc_auc 0.9266446857461058
pr_auc 0.9099899918671452


In [32]:
rusboost_model = RUSBoostClassifier(n_estimators=10, algorithm='SAMME.R', random_state=0)
rusboost_model.fit(X_train,y_train)
show_scores(X_val,y_val,rusboost_model)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.2

recall 0.9066666666666666
percision 0.0633147113594041
roc_auc 0.942276930291174
pr_auc 0.48342163987875547


In [33]:
# Gives value error on colab
# print("Random forest random undersampling")
# model = BalancedRandomForestClassifier(n_estimators=10)
# model.fit(X_train,y_train)
# show_scores(X_val,y_val,model)

In [34]:
print("Easy Ensemble")
easy_model = EasyEnsembleClassifier(n_estimators=10)
easy_model.fit(X_train,y_train)
show_scores(X_val,y_val,easy_model)

Easy Ensemble


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.2

recall 0.9066666666666666
percision 0.042821158690176324
roc_auc 0.936627833707009
pr_auc 0.4731748635441416


In [35]:
model_lis = [bagging_model,rf_model,rf_cls_weight_model,rf_bts_cls_weight_model,easy_model, rusboost_model]
model_names = ["BAGGING","RF","RF-CLSW", "RF-BTSTRP-CLSW", "EASY-ENSMBL", "RUSSBOOST"]

y_pred_lis = []
for model in model_lis:
  y_pred_lis.append(model.predict_proba(X_val)[:,1])

In [38]:
plot_pr(model_names, y_val,y_pred_lis)

In [39]:
plot_roc(model_names, y_val,y_pred_lis)